In [7]:
using Pkg
#Pkg.add("NBInclude")

Latex Macros:
$$
\newcommand{\E}{\text{E}}
\newcommand{\mbf}{\mathbf}
\newcommand{\bs}{\boldsymbol}
\newcommand{\Cov}{\text{Cov}}
\newcommand{\Var}{\text{Var}}
\newcommand{\Cov}{\text{Cov}}
\newcommand{\A}[1]{\mathbf{A}_{#1}}
\newcommand{\Ai}[1]{\mathbf{A}^{#1}}
$$

In [4]:
macro javascript_str(s) display("text/javascript", s); end
javascript"""
    MathJax.Hub.Config({
      TeX: { equationNumbers: { autoNumber: "AMS" } }
    });
    MathJax.Hub.Queue( 
        ["resetEquationNumbers",MathJax.InputJax.TeX], 
        ["PreProcess",MathJax.Hub], 
        ["Reprocess",MathJax.Hub] 
    );
"""

# Gibbs Sampling of Fixed and Random Effects

Consider the mixed linear model

\begin{equation}
\mathbf{y} = \mathbf{X}\boldsymbol{\beta} + \mathbf{Zu} + \mathbf{e}, 
\end{equation}

where $\boldsymbol{\beta}$ is a vector of fixed effects, $\mathbf{X}$ is an observed matrix that relates $\boldsymbol{\beta}$ to $\mathbf{y}$, $\mathbf{u}$ is a vector of multivariate normal random effects with null means and covariance matrix $\mathbf{G}\sigma^2_u$, and $\mathbf{e}$ is a vector of multivariate normal residuals with null means and covariance matrix $\mathbf{R}\sigma^2_e$. The fixed effects are usually assigned a flat prior distribution, and the matrices $\mathbf{G}$ and $\mathbf{R}$ are assumed to be known. The variance components $\sigma^2_u$ and $\sigma^2_e$ will be assigned scaled inverted chi-square prior distributions with scale parameter $S_{u}^{2}$ and degrees of freedom $\nu_{u}$ for $\sigma^2_u$ and parameters $S_{e}^{2}$ and $\nu_{e}$ for $\sigma^2_e$.

If we denote the location parameters ($\bs{\beta}, \mbf{u}$) of the model by $\bs{\theta}$, as we have seen before, the posteior distribution of $\bs{\theta}$ is multivariate normal with mean given by the solution to

$$
\mbf{C}\hat{\bs{\theta}} = \mbf{r}
$$

and the variance is $\mbf{C}^{-1}\sigma^2_e$,
where $\mbf{C}$ and $\mbf{r}$ are the coefficient matrix and right-hand-side of the MME. So, the quadratic in the exponent of this normal distribution is:



\begin{equation}
Q = \frac{-1}{2\sigma^2_e}(\bs{\theta} - \hat{\bs{\theta}})'\mathbf{C}(\bs{\theta} - \hat{\bs{\theta}}).
\label{eq:Q1}
\end{equation}

In order to use the single-site Gibbs sampler to construct a Markov chain for $\bs{\theta}$, we need to derive the full-conditional posterior distribution for $(\theta_i|\text{ELSE})$, where $\text{ELSE}$ stands for all other variables in the model other than $\theta_i$. To simplify this derivation, $\bs{\theta}$ is partitioned as:
$$
\bs{\theta} = [\bs{\theta}_1, \bs{\theta}_2],
$$
where $\bs{\theta}_1 = \theta_i$ and $\bs{\theta}_2$ contains the remaining elements of $\bs{\theta}$. Now, the quadratic (\ref{eq:Q1}) can be written as:

\begin{equation}
Q = \frac{-1}{2\sigma^2_e}
\begin{bmatrix}
(\bs{\theta}_1 - \hat{\bs{\theta}}_1) & (\bs{\theta}_2 - \hat{\bs{\theta}}_2)
\end{bmatrix}
\begin{bmatrix}
\mathbf{C}_{11} & \mathbf{C}_{12} \\
\mathbf{C}_{21} & \mathbf{C}_{22}
\end{bmatrix}
\begin{bmatrix}
(\bs{\theta}_1 - \hat{\bs{\theta}}_1) \\ 
(\bs{\theta}_2 - \hat{\bs{\theta}}_2)
\end{bmatrix},
\label{eq:Q2}
\end{equation}

where the coefficient matrix $\mbf{C}$ has also been partitioned to correspond to the partitioning of $\bs{\theta}$.

To derive the distribution of $(\bs{\theta}_1|\bs{\theta}_2)$, we need to only consider the terms in (\ref{eq:Q2}) that involve $\bs{\theta}_1$. These terms are (after dropping $\frac{-1}{2\sigma^2_e}$):


\begin{align}
Q_1 &=  \bs{\theta}_1'\mbf{C}_{11}\bs{\theta}_1 - 2\bs{\theta}_1'\mbf{C}_{11}\hat{\bs{\theta}}_1
    + 2\bs{\theta}_1'\mbf{C}_{12}\bs{\theta}_2 - 2\bs{\theta}_1'\mbf{C}_{12}\hat{\bs{\theta}}_2 \label{eq:Q_11} \\ 
    &= (\bs{\theta}_1 - \tilde{\bs{\theta}}_1)'\mbf{C}_{11}(\bs{\theta}_1 - \tilde{\bs{\theta}}_1) 
       - \tilde{\bs{\theta}}_1'\mbf{C}_{11}\tilde{\bs{\theta}}_1,
       \label{eq:Q_12}
\end{align}

where for (\ref{eq:Q_11}) to equal (\ref{eq:Q_12}), $\tilde{\bs{\theta}}_1$ must satisfy the equality: 
$$
\bs{\theta}_1'\mbf{C}_{11}\tilde{\bs{\theta}}_1 = \bs{\theta}_1'\mbf{C}_{11}\hat{\bs{\theta}}_1 
                                                       + \bs{\theta}_1'\mbf{C}_{12}\hat{\bs{\theta}}_2 
                                                       - \bs{\theta}_1'\mbf{C}_{12}\bs{\theta}_2.
$$
This equlity must hold for all $\bs{\theta}_1$, and so
\begin{align}
\mbf{C}_{11}\tilde{\bs{\theta}}_1 &= \mbf{C}_{11}\hat{\bs{\theta}}_1 
                                  + \mbf{C}_{12}\hat{\bs{\theta}}_2
                                  - \mbf{C}_{12}\bs{\theta}_2 \\
\mbf{C}_{11}\tilde{\bs{\theta}}_1 &= \mbf{r}_1 - \mbf{C}_{12}\bs{\theta}_2, \label{eq:singleSiteGibbs}                                                                   
\end{align}
because the right-hand-side of the MME corresponding to $\hat{\bs{\theta}}_1$ is equal to 

$$
\mbf{C}_{11}\hat{\bs{\theta}}_1 + \mbf{C}_{12}\hat{\bs{\theta}}_2 = \mbf{r}_1.
$$

Thus, $(\bs{\theta}_1|\bs{\theta}_2) \sim \text{N}(\tilde{\bs{\theta}},\mbf{C}_{11}^{-1}\sigma^2_e)$.

In [8]:
function Gibbs!(A,x,b,varRes::Float64,nIter::Int64;outFreq=100)
    n = size(x,1)
    xMean = zeros(n)
    for iter = 1:nIter
        if iter%outFreq==0
            println("at sample: ",iter)
        end
        for i=1:n
            cVarInv = 1.0/A[i,i]
            cMean   = cVarInv*(b[i] - A[:,i]'x) + x[i]
            x[i]    = randn()*sqrt(cVarInv*varRes) + cMean
        end
        xMean += (x - xMean)/iter
    end
    return xMean
end

Gibbs! (generic function with 1 method)

In [15]:
using NBInclude
@nbinclude("../MME/3.7.BuildMME.ipynb"; regex=r"#\s*EXECUTE")

getLhsRhs (generic function with 1 method)

In [16]:
pedigree = get_pedigree("simData.ped",separator=",",header=false);

The delimiter in simData.ped is ','.
Finished!


In [17]:
data = CSV.read("data.phen")

,Ind,Mat,y
,Int64,Int64,Float64
1,26,12,7.434
2,27,10,10.367
3,28,19,5.996
4,29,15,10.084
5,30,23,2.567
6,31,8,-0.198
7,32,21,12.163
8,33,16,0.833
9,34,17,2.39


In [18]:
using Statistics

In [19]:
var(data[:,3])

21.821335541313132

In [20]:
varGen = 10.0
varRes = 10.0
mme = initMME("y = intercept + Ind",varRes);
setRandom(mme,"Ind",varGen,pedigree);

In [21]:
lhs,rhs,names = getLhsRhs(mme,data);

In [22]:
x = zero(rhs);

In [25]:
nIter = 50000
solGibbs = Gibbs!(lhs,x,rhs,varRes,nIter;outFreq=1000);

at sample: 1000
at sample: 2000
at sample: 3000
at sample: 4000
at sample: 5000
at sample: 6000
at sample: 7000
at sample: 8000
at sample: 9000
at sample: 10000
at sample: 11000
at sample: 12000
at sample: 13000
at sample: 14000
at sample: 15000
at sample: 16000
at sample: 17000
at sample: 18000
at sample: 19000
at sample: 20000
at sample: 21000
at sample: 22000
at sample: 23000
at sample: 24000
at sample: 25000
at sample: 26000
at sample: 27000
at sample: 28000
at sample: 29000
at sample: 30000
at sample: 31000
at sample: 32000
at sample: 33000
at sample: 34000
at sample: 35000
at sample: 36000
at sample: 37000
at sample: 38000
at sample: 39000
at sample: 40000
at sample: 41000
at sample: 42000
at sample: 43000
at sample: 44000
at sample: 45000
at sample: 46000
at sample: 47000
at sample: 48000
at sample: 49000
at sample: 50000


In [26]:
QRLhs = qr(lhs) 
sol = QRLhs\rhs
[names solGibbs sol]

126×3 Array{Any,2}:
 "intercept"   5.3121       5.31345  
 "Ind: 5"      2.35044      2.37165  
 "Ind: 21"     1.64543      1.61821  
 "Ind: 32"     3.61474      3.61314  
 "Ind: 15"     0.572928     0.571631 
 "Ind: 29"     2.56382      2.57128  
 "Ind: 1"     -1.80097     -1.82355  
 "Ind: 4"      0.882558     0.891078 
 "Ind: 20"     3.25091      3.24584  
 "Ind: 54"     1.64348      1.64616  
 "Ind: 22"    -3.25768     -3.22863  
 "Ind: 78"    -0.635763    -0.621334 
 "Ind: 81"    -0.675892    -0.696458 
 ⋮                                   
 "Ind: 45"     2.53564      2.54842  
 "Ind: 68"    -2.28953     -2.30608  
 "Ind: 56"    -0.215254    -0.212933 
 "Ind: 116"    1.72992      1.70612  
 "Ind: 36"    -2.67832     -2.67382  
 "Ind: 66"    -1.14836     -1.14998  
 "Ind: 76"    -5.09805     -5.10858  
 "Ind: 95"     0.00718447   0.0174043
 "Ind: 85"     3.20232      3.19323  
 "Ind: 48"    -0.812982    -0.814834 
 "Ind: 103"    1.14704      1.1539   
 "Ind: 30"    -0.119004    -0.

In [27]:
cor(ans[:,2:3])

2×2 Array{Float64,2}:
 1.0       0.999988
 0.999988  1.0     